# Obstacle Avoidance 
**Full Coding At [Obstacle_Avoidance.py](Obstacle_Avoidance.py)** 
<br>
This is the documentation for Obstacle Avoidance (With Camera) 

***Libraries Used In The Script***
- Ultrasonic_sens  
- PCA9685_MC 
- Motor_Encoder  
- time 
- threading 
- picamera2 
- OpenCV
- libcamera 
<br>



**All the libraries are custom made libraraies thus installation is no needed**
- *Ultrasonic_sens* 
    - Used to obtained all the value of the ultrasonic ranger at once
- *PCA9685_MC* 
    - Used to controll the Motor Direction more easily using Servo Motor HAT 
- *Motor_Encoder* 
     - Used to obtained the value of the RPM of the motor (With OLED Function)
- *time*
    - Used for delay 

## Let's Start Coding ! 
### 1. Import all the Libraries 
- Ultrasonic_sens  
- PCA9685_MC 
- Motor_Encoder  
- time 

In [ ]:
from Ultrasonic_sens import Ultrasonic 
from PCA9685_MC import Motor_Controller
from Motor_Encoder import Encoder 
import time 


### 2. Initialise the libraries in the *init* Function 
- Create global variable so that other function can also access to it. 
  ```python 
  global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist
  ```
- Make sure the initaliation process is done 
  ``` python 
  isInit = False 
  ```
  
- If the initialisation is not **True**(Not be initialise) run the following:
  - Initialise the motor encoder library with the **ODISPLAY** enabled 
  - Initialise the ultrasonic sensors 
  - Initialise the motor library 
  - Set the speed at 20 
  - set the rotation speed  at 15 
  - Set the threshold to 30 
  - Set the minimum threshold distance to 10 
  - Set the initialation flag to **True**  

- If the initialation process is done, skip this function. 
  

In [ ]:
def init():
    global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist
    isInit = False

    if not isInit: 
        enc = Encoder(ODISPLAY=True)
        ultrasonic = Ultrasonic()
        Motor = Motor_Controller()
        Speed = 20
        rotation_speed = 15
        threshold = 30 
        min_thresh_dist = 10

        isInit = True 
    else:
        pass

### 3. The *Obstacle Avoid Function* `def obstacle_avoid(left. front. right)` 
- This is the logic part that wil process the data get from the sensors and determine the diurection of the obstacle. 
- In this function the function will collect the data from the left, front and right ultrasonic sensor to determine the distance of the obstacle from the sourounding. 
- Access to the global variables 
  ``` python
  global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist
  ```
- If the value of the fron sensor is less than the given threshold (30) Multiple condition will be triggred. 
  - If the front value is less than the minimum threshold value (20), the Car will go backward with the speed of 20 
  - If the left and right sensor are less than the minumum vlue, the Car will Reverse untill the values re more than minimum threshold value (20)
  - If obstacle at the left ensor is less than the threshold value, the car will turn right. 
  - If the obstacle at the right sensor is less than the threshold value, the car will turn left. 
  - Else, the Car will brake. 

- Else if the left sensor is less than the threshold value but the front sensor still have some sapce,  the car will turn Right
- Else if the  right sensor is less than the threshold value but the front sensor still have some space, the car will turn left 
- **The two else if logic above is trying to make sure the car is in the center if two obstacle*

- If the left and the right sensor are all less than threshold but the foront sensor is more than the threshold value, the care will proceed forward.  
  
- If all the condition above is not meet, the car will brake 

In [ ]:
def obstacle_Avoid(left, front, right):
    global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist
    
    if front < threshold:
        if front <= min_thresh_dist:
            Motor.Backward(Speed)
            time.sleep(0.1)
            Motor.Brake()
        elif left < min_thresh_dist and right < min_thresh_dist:
            Motor.Backward(Speed)
            time.sleep(0.1)
            Motor.Brake()
        elif left < threshold:
            Motor.Clock_Rotate(rotation_speed)
            time.sleep(0.5)
            Motor.Brake()
        elif right < threshold:
            Motor.AntiClock_Rotate(rotation_speed)
            time.sleep(0.5)
            Motor.Brake()
        else:
            Motor.Backward(Speed)
            time.sleep(0.1)
            Motor.Brake()
    
    elif left < threshold:
        Motor.Clock_Rotate(rotation_speed)
        time.sleep(1)
        Motor.Brake()
    
    elif right < threshold:
        Motor.AntiClock_Rotate(rotation_speed)  
        time.sleep(1)
        Motor.Brake()
    
    elif right < threshold and left < threshold and front > threshold :
            Motor.Forward(Speed)
            time.sleep(0.1)
            Motor.Brake()
    else:
        Motor.Brake()

### 4. The `main` function 
- Get the global variables by `global` 
    ``` python
    global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist
    ```

- Coll for the `init()` function to makesure all theh libraries are initialised. 

- `While True` is a technique to create a loop in python. 
    - Obtain the encoder value. if the **ODISPLAY** is true, the encoder value wil be display on the OLED. 
    - Display the value of the left, front and right ultrasonic sensor 
    - To make sure the variable that holds the sensor's value is valid, the if statem,ent will checkis the value is None or a valud value.
        ```python 
        if (left and front and right is not None) 
        ``` 
    - If the value from the sensor is not valid *(None)* 
        - The Car will brake and wait for the value of the sensor to become valid (not None). 
    - This step is to prevent the car moving without detecting the obstacle from the enviroment. 
    

In [ ]:
def main(): 
    global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist
    init()
    while True:
        enc.encoder()
        left, front, right = ultrasonic.distances()
        time.sleep(0.1)
        if (left and front and right is not None): 
            print("Left: {:.2f} ".format(left)) 
            print("Front: {:.2f}".format(front))
            print("Right: {:.2f}".format(right)) 
            print(" ")
            obstacle_Avoid(left, front, right)
            time.sleep(0.1)
        else:
            print("No data received")
            Motor.Brake()
            time.sleep(1)
        

### 5. Wraping Up 
- If the program is started, run the `main()` function 
    ```py
    try:
        if __name__ == "__main__":
            main()
    ```
- Keyboard Inturrupt is detected (Ctrl + C) 
    ```py 
    except KeyboardInterrupt:
        Motor.Brake()
        enc.stop()
        print("Program Terminated")
    ```
- When the programe ends (Finally)
    ```py
    finally:
        Motor.cleanup()
        enc.stop()
        print("Program Terminated")
    ```



In [ ]:
try:
    if __name__ == "__main__":
        main()

except KeyboardInterrupt:
    Motor.Brake()
    enc.stop()
    print("Program Terminated")
finally:
    Motor.cleanup()
    enc.stop()
    print("Program Terminated")
